<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/sensors_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

DBLink = "https://cloud-project-5adfc-default-rtdb.europe-west1.firebasedatabase.app/"

from firebase import firebase

FBconn = firebase.FirebaseApplication(DBLink, None)

# Function to fetch data from Firebase for a specific time range
def fetch_data_from_firebase(full_start_time_str):
    try:
        start_time = datetime.strptime(full_start_time_str, '%Y-%m-%d %H:%M:%S')
    except ValueError as e:
        print(f"Error parsing start time string '{full_start_time_str}': {e}")
        return pd.DataFrame()

    end_time = start_time + timedelta(hours=1)

    all_data_original = FBconn.get('/sensor_readings', None)

    all_data = {}
    # marge indoor and outdoor dict
    for d in (all_data_original['indoor'], all_data_original['outdoor']):
        for key, value in d.items():
            if key not in all_data:
                all_data[key] = {}
            for k,v in value.items():
              if v == 'N/A':
                all_data[key][k] = None
              else:
                all_data[key][k] = v

    if not all_data:
        return pd.DataFrame()

    filtered_data = {}
    for timestamp_str, data in all_data.items():
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
            if start_time <= timestamp < end_time:
                filtered_data[timestamp] = data
        except ValueError:
            print(f"Skipping entry with invalid timestamp in Firebase: {timestamp_str}")
            pass

    if not filtered_data:
        return pd.DataFrame()

    df = pd.DataFrame.from_dict(filtered_data, orient='index')
    if not df.empty: # Ensure index is DatetimeIndex for plotting if df is not empty
        df.index = pd.to_datetime(df.index)
        df.index.name = 'Timestamp'
        df = df.sort_index()
    return df

# Helper function to create plots (reduces redundancy)
def create_sensor_plot(df, column_name, title, ylabel):
    plt.figure(figsize=(10, 4))
    if column_name in df.columns and not df[column_name].empty:
        plt.plot(df.index, df[column_name])
    else:
        plt.text(0.5, 0.5, f'No data for {column_name}', horizontalalignment='center', verticalalignment='center')
    plt.grid(True) # Add grid
    plt.title(title, fontsize=18)
    plt.xlabel('Timestamp')
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plot_fig = plt.gcf()
    plt.close()
    return plot_fig

# Function to generate plots for the selected time range
def generate_plots(selected_year, selected_month, selected_day, selected_hour_str):
    if not selected_year or not selected_month or not selected_day or not selected_hour_str:
        print("Date or hour not fully selected.")
        return None, None, None, None, None, None, None

    try:
        selected_date_str = f"{selected_year}-{int(selected_month):02d}-{int(selected_day):02d}"
        selected_hour_int = int(selected_hour_str)
        full_start_time_str = f"{selected_date_str} {selected_hour_int:02d}:00:00"
    except ValueError:
        print(f"Invalid date or hour format: Year {selected_year}, Month {selected_month}, Day {selected_day}, Hour {selected_hour_str}")
        return None, None, None, None, None, None, None

    df_hour = fetch_data_from_firebase(full_start_time_str)

    if df_hour.empty:
        print(f"No data found for {full_start_time_str}")
        return None, None, None, None, None, None, None

    plot_dlight = create_sensor_plot(df_hour, 'Outdoor Dlight', 'Outdoor Dlight', 'Dlight (Lux)')
    plot_out_temp = create_sensor_plot(df_hour, 'Outdoor Temperature', 'Outdoor Temperature', 'Temp (°C)')
    plot_out_humidity = create_sensor_plot(df_hour, 'Outdoor Humidity', 'Outdoor Humidity', 'Humidity (%)')
    plot_in_humidity = create_sensor_plot(df_hour, 'Indoor Humidity', 'Indoor Humidity', 'Humidity (%)')
    plot_in_temp = create_sensor_plot(df_hour, 'Indoor Temperature', 'Indoor Temperature', 'Temp (°C)')
    plot_pressure = create_sensor_plot(df_hour, 'Indoor Pressure', 'Indoor Pressure', 'Pressure (hPa)')
    plot_distance = create_sensor_plot(df_hour, 'Indoor Distance', 'Indoor Distance', 'Distance (mm)')

    return plot_dlight, plot_out_temp, plot_out_humidity, plot_in_humidity, plot_in_temp, plot_pressure, plot_distance

def get_all_data_keys():
    """
    Retrieves and merges sensor data from Firebase for both indoor and outdoor sources.
    Replaces 'N/A' values with None for easier processing.

    Returns:
        dict: Merged dictionary of all timestamped sensor readings.
    """
    all_data_keys_original = FBconn.get('/sensor_readings', None)
    all_data_keys = {}
    if all_data_keys_original:
        # Merge indoor and outdoor dicts
        for d in (all_data_keys_original.get('indoor', {}), all_data_keys_original.get('outdoor', {})):
            for key, value in d.items():
                if key not in all_data_keys:
                    all_data_keys[key] = {}
                for k, v in value.items():
                    all_data_keys[key][k] = None if v == 'N/A' else v
    return all_data_keys

def get_available_timestamps_str():
    """
    Returns a sorted list of all timestamps (as strings) present in the merged sensor data.
    """
    all_data_keys = get_all_data_keys()
    return sorted(all_data_keys.keys()) if all_data_keys else []

def get_unique_date_strings():
    """
    Returns a sorted list of unique date strings in the format 'YYYY-MM-DD' extracted from all available timestamps.
    """
    available_timestamps_str = get_available_timestamps_str()
    return sorted(list(set([
        datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
        for ts in available_timestamps_str
    ])))

def get_available_years():
    """
    Returns a sorted list of unique years extracted from all available timestamps.
    """
    available_timestamps_str = get_available_timestamps_str()
    return sorted(list(set([
      datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').year
      for ts in available_timestamps_str
    ])))

def get_date_to_hours_map():
    """
    Builds a mapping from each unique date (as string) to a list of available hour strings for that date.
    """
    available_timestamps_str = get_available_timestamps_str()
    date_to_hours_map = {}
    for ts_str in available_timestamps_str:
        dt_obj = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S')
        date_key_str = dt_obj.strftime('%Y-%m-%d')
        hour_val = dt_obj.hour
        if date_key_str not in date_to_hours_map:
            date_to_hours_map[date_key_str] = []
        if hour_val not in date_to_hours_map[date_key_str]:
            date_to_hours_map[date_key_str].append(hour_val)
    for date_key_str in date_to_hours_map:
        date_to_hours_map[date_key_str].sort()
        date_to_hours_map[date_key_str] = [str(h) for h in date_to_hours_map[date_key_str]]
    return date_to_hours_map

# Function to get month choices (name, value) for a given year
def get_month_choices_for_year(year):
    if year is None or year not in get_date_to_month_map():
        return []
    available_months = get_date_to_month_map()[year]
    return [(calendar.month_name[month], month) for month in available_months]

def get_date_to_month_map():
    # Create a mapping from year to a list of unique months available in that year
    available_timestamps_str = get_available_timestamps_str()
    year_to_months_map = {}
    for ts_str in available_timestamps_str:
        dt_obj = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S')
        year = dt_obj.year
        month = dt_obj.month
        if year not in year_to_months_map:
            year_to_months_map[year] = []
        if month not in year_to_months_map[year]:
            year_to_months_map[year].append(month)

    # Sort the months for each year
    for year in year_to_months_map:
        year_to_months_map[year].sort()

    return year_to_months_map
